## Dataset downsampling and format
### To test:
1. Compare size of ogg and mp3 files
2. Check how quickly ogg and mp3 are opened (decompressed)
3. Downsampling to 32 kHz (compare size)

### Why?
1. ogg is used in all birdclef competition on kaggle
2. ogg is promising to be smaller while not loosing quality

## Sample 100 recordings

In [1]:
import os
from birdclassification.preprocessing.filtering import initial_filter
from birdclassification.preprocessing.utils import timer
import shutil

SOURCE = '/media/jacek/E753-A120/xeno-canto/'
DST_MP3 = '/home/jacek/Documents/mp3/'
DST_OGG = '/home/jacek/Documents/ogg/'

In [2]:
df = initial_filter()
sample = df.sample(n = 100, random_state=123)
sample['filepath'] = df.apply(lambda x: f"{SOURCE}{str(x['id'])}.mp3" , axis=1)
sample

/home/jacek/PycharmProjects/Bird-classification-model/birdclassification/preprocessing/filtering.py:16: DtypeWarning: Columns (10,39,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/xeno_canto_recordings.csv')


,Unnamed: 0,id,gen,sp,ssp,group,en,rec,cnt,loc,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Latin name,filepath
134932,134932,708143,Alauda,arvensis,NaN,birds,Eurasian Skylark,Georg Amann,Austria,"Rankweil, Feldkirch, Vorarlberg",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alauda arvensis,/media/jacek/E753-A120/xeno-canto/708143.mp3
89233,89233,619297,Jynx,torquilla,torquilla,birds,Eurasian Wryneck,Karol Łanocha,Poland,"Gmina Kęty (near Kęty), Powiat oświęcimski, M...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jynx torquilla,/media/jacek/E753-A120/xeno-canto/619297.mp3
190242,190242,391049,Sitta,europaea,NaN,birds,Eurasian Nuthatch,Lars Edenius,Sweden,"Baggböle, Umeå, Västerbottens län",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sitta europaea,/media/jacek/E753-A120/xeno-canto/391049.mp3
70153,70153,701523,Nycticorax,nycticorax,NaN,birds,Black-crowned Night Heron,Dominique Guillerme,France,"Chassey, Côte-d'Or, Bourgogne-Franche-Comté",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nycticorax nycticorax,/media/jacek/E753-A120/xeno-canto/701523.mp3
7051,7051,590644,Bucephala,clangula,NaN,birds,Common Goldeneye,Simon Elliott,United Kingdom,"Highland, Scotland",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucephala clangula,/media/jacek/E753-A120/xeno-canto/590644.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171045,171045,564408,Aegithalos,caudatus,NaN,birds,Long-tailed Tit,Falco,France,"Montgardin, Hautes-Alpes, Provence-Alpes-Côte ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aegithalos caudatus,/media/jacek/E753-A120/xeno-canto/564408.mp3
107631,107631,87701,Coloeus,monedula,NaN,birds,Western Jackdaw,Volker Arnold,Germany,"Heide-Ruesdorf, Dithmarschen, Schleswig-Holstein",...,44100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coloeus monedula,/media/jacek/E753-A120/xeno-canto/87701.mp3
242467,242467,284310,Anthus,pratensis,NaN,birds,Meadow Pipit,Joost van Bruggen,Netherlands,"Arnhem, Arnhem, Gelderland",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anthus pratensis,/media/jacek/E753-A120/xeno-canto/284310.mp3
10809,10809,183095,Anas,platyrhynchos,NaN,birds,Mallard,Albert Lastukhin,Kazakhstan,"Shieli District, Kyzylorda Province",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anas platyrhynchos,/media/jacek/E753-A120/xeno-canto/183095.mp3


## Copy sample of 100 files to a folder

In [3]:
for filepath in sample['filepath']:
    shutil.copy(filepath, DST_MP3 + filepath.split(sep = '/')[-1])

## Convert to .ogg

In [4]:
import os
from pydub import AudioSegment

src = DST_MP3 
dst = DST_OGG

files = os.listdir(src)
for file in files:
    name = file.split(".")[0]
    if name != '':
        try:
            sound = AudioSegment.from_file(f"{src}/{name}.mp3")
            sound.export(f"{dst}/{name}.mp3", format="mp3")
        except:
            print("Error")

#### Time of execution: 3 minutes, 3 seconds
#### ogg folder size: 76 MB
#### mp3 folder size: 224,4MB

In [6]:
import librosa
import os
from birdclassification.preprocessing.utils import timer

@timer
def read_ogg():
    file_list = os.listdir(DST_OGG)
    for file in file_list:
        filepath = DST_OGG + file
        audio, sr = librosa.load(filepath)

@timer
def read_mp3():
    file_list = os.listdir(DST_MP3)
    for file in file_list:
        filepath = DST_MP3 + file
        audio, sr = librosa.load(filepath)

In [32]:
read_mp3()

Note: Illegal Audio-MPEG-Header 0x494f4e00 at offset 397500.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x52450050 at offset 1271767.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


Function: 'read_mp3', execution time: 22.7945396900177


In [33]:
read_ogg()

Function: 'read_ogg', execution time: 25.552635192871094


In [7]:
read_mp3()

Function: 'read_mp3', execution time: 16.1896390914917


In [ ]:
audio = AudioSegment.from_file('Canon.mp3')
audio.export('Canon2.mp3', bitrate='128k', format='mp3')